In [2]:
import numpy as np
import pandas as pd
import gym
#import logz_pytorch as logz
import logz
import scipy.signal
import os
import time
import inspect

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

#============================================================================================#
# Utilities
#============================================================================================#


def pathlength(path):
    return len(path["reward"])


class MLP(nn.Module):

    # ========================================================================================#
    #                           ----------SECTION 3----------
    # Network building
    #
    # Your code should make a feedforward neural network (also called a multilayer perceptron)
    # with 'n_layers' hidden layers of size 'size' units.
    #
    # The output layer should have size 'output_size' and activation 'output_activation'.
    #
    # ========================================================================================#

    def __init__(self, input_size, output_size, n_layers=2, size=64, activation=F.tanh, output_activation=None):
        super(MLP, self).__init__()
        self.dropout_rate = 0.3
        self.fc1 = nn.Linear(input_size, size)
        self.fc2 = nn.Linear(size, size)
        self.fc3 = nn.Linear(size, output_size)
#         self.fc = (
#             [nn.Linear(input_size, size)] + 
#             [nn.Linear(size, size)] * n_layers + 
#             [nn.Linear(size, output_size)]
#         )
        self.drop = nn.Dropout(p = self.dropout_rate)
        self.activation = activation
        self.output_activation = output_activation

    # assert output_activation is None, 'output activation must be None, other options not implemented'
 
    def forward(self, x):
        x = self.activation(self.fc1(x))  
#        x = self.activation(self.fc2(x))  
        if self.output_activation is None:
            x = self.fc3(x)                      # Last layer 
        else:
            x = self.output_activation(self.fc3(x))  
        return x.view(x.size(0), -1)

    def set_dropout_rate(self, p):
        self.dropout_rate = p
        

#============================================================================================#
# Policy Gradient
#============================================================================================#

def train_PG(exp_name='',
             env_name='CartPole-v0',
             n_iter=100,
             gamma=1.0,
             min_timesteps_per_batch=1000,
             max_path_length=None,
             learning_rate=5e-3,
             reward_to_go=True,
             animate=True,
             logdir=None,
             normalize_advantages=True,
             nn_baseline=False,
             seed=0,
             # network arguments
             n_layers=1,
             size=32
             ):

    start = time.time()

    # Configure output directory for logging
    logz.configure_output_dir(logdir)
    logz.G.first_row = True
    # Log experimental parameters
    #args = inspect.getfullargspec(train_PG)[0]
    args = inspect.getargspec(train_PG)[0]
    locals_ = locals()
    params = {k: locals_[k] if k in locals_ else None for k in args}
    logz.save_params(params)

    # Set random seeds
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Make the gym environment
    env = gym.make(env_name)

    # Is this env continuous, or discrete?
    discrete = isinstance(env.action_space, gym.spaces.Discrete)

    # Maximum length for episodes
    max_path_length = max_path_length or env.spec.max_episode_steps

    #========================================================================================#
    # Notes on notation:
    #
    # Symbolic variables have the prefix sy_, to distinguish them from the numerical values
    # that are computed later in the function
    #
    # Prefixes and suffixes:
    # ob - observation
    # ac - action
    # _no - this tensor should have shape (batch size /n/, observation dim)
    # _na - this tensor should have shape (batch size /n/, action dim)
    # _n  - this tensor should have shape (batch size /n/)
    #
    # Note: batch size /n/ is defined at runtime, and until then, the shape for that axis
    # is None
    #========================================================================================#

    # Observation and action sizes
    ob_dim = env.observation_space.shape[0]
    ac_dim = env.action_space.n if discrete else env.action_space.shape[0]

    mlp = MLP(input_size = ob_dim, output_size = ac_dim, 
              n_layers = n_layers, size = size,output_activation=None)
#     loss = TODO # Loss function that we'll differentiate to get the policy gradient.
    update_op = optim.Adam(mlp.parameters(), lr=learning_rate)


    #========================================================================================#
    #                           ----------SECTION 5----------
    # Optional Baseline
    #========================================================================================#

    if nn_baseline:
        mlp_baseline = MLP(ob_dim, 1, n_layers=n_layers, size=size,output_activation=None)
        update_baseline_op = optim.Adam(mlp_baseline.parameters(), lr=learning_rate)
        q_prev_mean, q_prev_std = 0., 1.


    #========================================================================================#
    # Training Loop
    #========================================================================================#

    total_timesteps = 0

    for itr in range(n_iter):
        print("********** Iteration %i ************"%itr)

        # Collect paths until we have enough timesteps
        timesteps_this_batch = 0
        paths = []
        while True:
            ob = env.reset()
            obs, acs, rewards = [], [], []
            animate_this_episode=(len(paths)==0 and (itr % 10 == 0) and animate)
            steps = 0
            while True:
                if animate_this_episode:
                    env.render()
                    time.sleep(0.05)
                obs.append(ob)

                #TODO: CHECK_IT!
#                 ac_logits = mlp(Variable(torch.Tensor(ob[None]))).data.numpy()


                mu = mlp(Variable(torch.Tensor(ob[None]))).data.numpy()[0][0]
                std = 0.01
        
        
                ac_raw = np.random.randn()
                ac = ac_raw * std + mu 
                # Pick action according to mlp policy. mlp output is actions logits

#                 ac_probs = 1. / (1 + np.exp( -ac_logits))
#                 ac_probs = ac_probs / ac_probs.sum()

#                 ac = np.random.choice(range(ac_dim), p = ac_logits)
                
                #ac = ac[0]
                acs.append(ac)
                #1/0
                ob, rew, done, _ = env.step([ac])
                rewards.append(rew)
                steps += 1
                if done or steps > max_path_length:
                    break
            path = {"observation" : np.array(obs),
                    "reward" : np.array(rewards),
                    "action" : np.array(acs)}
            paths.append(path)
            timesteps_this_batch += pathlength(path)
            if timesteps_this_batch > min_timesteps_per_batch:
                break
        total_timesteps += timesteps_this_batch

        # Build arrays for observation, action for the policy gradient update by concatenating
        # across paths
        ob_no = np.concatenate([path["observation"] for path in paths])
        ac_na = np.concatenate([path["action"] for path in paths])

 
        # YOUR_CODE_HERE
        if reward_to_go:
            q_n = np.concatenate([
                                    np.cumsum([r * gamma ** i for i,r in enumerate(path["reward"])][::-1])[::-1]                
                                 for path in paths])
        else:            
            q_n = np.concatenate([
                                    [sum([r * gamma ** i for i,r in enumerate(path["reward"])])] 
                                        * len(path["reward"]) 
                                 for path in paths])
            
        #====================================================================================#
        #                           ----------SECTION 5----------
        # Computing Baselines
        #====================================================================================#

        if nn_baseline:
            # If nn_baseline is True, use your neural network to predict reward-to-go
            # at each timestep for each trajectory, and save the result in a variable 'b_n'
            # like 'ob_no', 'ac_na', and 'q_n'.
            #
            # Hint #bl1: rescale the output from the nn_baseline to match the statistics
            # (mean and std) of the current or previous batch of Q-values. (Goes with Hint
            # #bl2 below.)
            v_n = mlp_baseline(Variable(torch.Tensor(ob_no)))[:,0]
            b_n = v_n.data.numpy() * q_prev_std + q_prev_mean
            adv_n = q_n - b_n
            q_prev_mean, q_prev_std = q_n.mean(),q_n.std()
            
        else:
            adv_n = q_n.copy()

        #====================================================================================#
        #                           ----------SECTION 4----------
        # Advantage Normalization
        #====================================================================================#

        if normalize_advantages:
            # On the next line, implement a trick which is known empirically to reduce variance
            # in policy gradient methods: normalize adv_n to have mean zero and std=1.
            # YOUR_CODE_HERE
            adv_n = (adv_n - adv_n.mean())/(adv_n.std() + 1e-15)


        #====================================================================================#
        #                           ----------SECTION 5----------
        # Optimizing Neural Network Baseline
        #====================================================================================#

        if nn_baseline:
            # ----------SECTION 5----------
            # If a neural network baseline is used, set up the targets and the inputs for the
            # baseline.
            #
            # Fit it to the current batch in order to use for the next iteration. Use the
            # baseline_update_op you defined earlier.
            #
            # Hint #bl2: Instead of trying to target raw Q-values directly, rescale the
            # targets to have mean zero and std=1. (Goes with Hint #bl1 above.)

            baseline_loss = ((v_n - Variable(torch.Tensor((q_n-q_n.mean())/q_n.std())))**2).mean()
            update_baseline_op.zero_grad()
            baseline_loss.backward()
            update_baseline_op.step()
            

        #====================================================================================#
        #                           ----------SECTION 4----------
        # Performing the Policy Update
        #====================================================================================#

        # Call the update operation necessary to perform the policy gradient update based on
        # the current batch of rollouts.
        #
        # For debug purposes, you may wish to save the value of the loss function before
        # and after an update, and then log them below.

#         PG = E_{tau} [sum_{t=0}^T grad log pi(a_t|s_t) * (Q_t - b_t )]
        # YOUR_CODE_HERE

        #Another solution
#         a,ac,a.view(-1)[torch.LongTensor(ac+3*np.arange(20))]
        adv_var = torch.autograd.Variable(torch.FloatTensor(adv_n) , requires_grad=False)
    
#         actions_t = torch.LongTensor(ac_na+ac_dim*np.arange(ob_no.shape[0]))         
        PGI = (((((mlp(Variable(torch.Tensor(ob_no)))[:,0] - Variable(torch.Tensor(ac_na)))/std)**2) 
                * adv_var).sum() 
               / len(paths)
              )
        print('avg action {}. abs {}'.format(ac_na.mean(),np.abs(ac_na).mean()))
        update_op.zero_grad()
        PGI.backward()
        update_op.step()

        
        
        # Log diagnostics
        returns = [path["reward"].sum() for path in paths]
        ep_lengths = [pathlength(path) for path in paths]
        logz.log_tabular("Time", time.time() - start)
        logz.log_tabular("Iteration", itr)
        logz.log_tabular("AverageReturn", np.mean(returns))
        logz.log_tabular("StdReturn", np.std(returns))
        logz.log_tabular("MaxReturn", np.max(returns))
        logz.log_tabular("MinReturn", np.min(returns))
        logz.log_tabular("EpLenMean", np.mean(ep_lengths))
        logz.log_tabular("EpLenStd", np.std(ep_lengths))
        logz.log_tabular("TimestepsThisBatch", timesteps_this_batch)
        logz.log_tabular("TimestepsSoFar", total_timesteps)
        logz.dump_tabular()


def main():
    import argparse
#     parser = argparse.ArgumentParser()
#     parser.add_argument('env_name', type=str)
#     parser.add_argument('--exp_name', type=str, default='vpg')
#     parser.add_argument('--render', action='store_true')
#     parser.add_argument('--discount', type=float, default=1.0)
#     parser.add_argument('--n_iter', '-n', type=int, default=100)
#     parser.add_argument('--batch_size', '-b', type=int, default=1000)
#     parser.add_argument('--ep_len', '-ep', type=float, default=-1.)
#     parser.add_argument('--learning_rate', '-lr', type=float, default=5e-3)
#     parser.add_argument('--reward_to_go', '-rtg', action='store_true')
#     parser.add_argument('--dont_normalize_advantages', '-dna', action='store_true')
#     parser.add_argument('--nn_baseline', '-bl', action='store_true')
#     parser.add_argument('--seed', type=int, default=1)
#     parser.add_argument('--n_experiments', '-e', type=int, default=1)
#     parser.add_argument('--n_layers', '-l', type=int, default=1)
#     parser.add_argument('--size', '-s', type=int, default=32)
#     args = parser.parse_args()

    args = pd.Series()
    args.batch_size=1000
    args.discount=0.99
    args.dont_normalize_advantages=True
    args.env_name='Pendulum-v0'
    args.ep_len=-1.0
    args.exp_name='lr_1e-2_rtg'
    args.learning_rate=1e-3
    args.n_experiments=4
    args.n_iter=100
    args.n_layers=1
    args.nn_baseline=False
    args.render=False
    args.reward_to_go=True
    args.seed=1
    args.size_=64

                
    if not(os.path.exists('data')):
        os.makedirs('data')
    logdir = args.exp_name + '_' + args.env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
    logdir = os.path.join('data', logdir)
    if not(os.path.exists(logdir)):
        os.makedirs(logdir)

    max_path_length = args.ep_len if args.ep_len > 0 else None

    for e in range(args.n_experiments):
        seed = args.seed + 10*e
        print('Running experiment with seed %d'%seed)
        def train_func():
            train_PG(
                exp_name=args.exp_name,
                env_name=args.env_name,
                n_iter=args.n_iter,
                gamma=args.discount,
                min_timesteps_per_batch=args.batch_size,
                max_path_length=max_path_length,
                learning_rate=args.learning_rate,
                reward_to_go=args.reward_to_go,
                animate=args.render,
                logdir=os.path.join(logdir,'%d'%seed),
                normalize_advantages=not(args.dont_normalize_advantages),
                nn_baseline=args.nn_baseline,
                seed=seed,
                n_layers=args.n_layers,
                size=args.size_
                )
        train_func()


if __name__ == "__main__":
    main()

/home/osminin/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:98: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
[2017-12-12 00:26:25,107] Making new env: Pendulum-v0


Running experiment with seed 1
Logging data to data/lr_1e-2_rtg_Pendulum-v0_12-12-2017_00-26-25/1/log.txt
********** Iteration 0 ************
avg action 0.08570540503243124. abs 0.30817467725076764
----------------------------------------
|               Time |           0.594 |
|          Iteration |               0 |
|      AverageReturn |       -1.41e+03 |
|          StdReturn |             258 |
|          MaxReturn |       -1.17e+03 |
|          MinReturn |       -1.91e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |         1.2e+03 |
|               Time |           0.594 |
|          Iteration |               0 |
|      AverageReturn |       -1.41e+03 |
|          StdReturn |             258 |
|          MaxReturn |       -1.17e+03 |
|          MinReturn |       -1.91e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch | 

avg action -0.030543476301173135. abs 0.4082361320037063
----------------------------------------
|               Time |             2.4 |
|          Iteration |               4 |
|      AverageReturn |       -1.14e+03 |
|          StdReturn |            46.3 |
|          MaxReturn |       -1.05e+03 |
|          MinReturn |        -1.2e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |           6e+03 |
|               Time |             2.4 |
|          Iteration |               4 |
|      AverageReturn |       -1.14e+03 |
|          StdReturn |            46.3 |
|          MaxReturn |       -1.05e+03 |
|          MinReturn |        -1.2e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |           6e+03 |
|               Time |             2.4 |
|          Iteration |               4 |


avg action 0.19901230479329202. abs 0.486448134014553
----------------------------------------
|               Time |            4.14 |
|          Iteration |               8 |
|      AverageReturn |       -1.16e+03 |
|          StdReturn |            45.2 |
|          MaxReturn |       -1.08e+03 |
|          MinReturn |       -1.21e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        1.08e+04 |
|               Time |            4.14 |
|          Iteration |               8 |
|      AverageReturn |       -1.16e+03 |
|          StdReturn |            45.2 |
|          MaxReturn |       -1.08e+03 |
|          MinReturn |       -1.21e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        1.08e+04 |
|               Time |            4.14 |
|          Iteration |               8 |
|  

avg action -0.0028148397021520682. abs 0.5295391851388493
----------------------------------------
|               Time |            5.94 |
|          Iteration |              12 |
|      AverageReturn |       -1.19e+03 |
|          StdReturn |            42.5 |
|          MaxReturn |       -1.15e+03 |
|          MinReturn |       -1.27e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        1.56e+04 |
|               Time |            5.94 |
|          Iteration |              12 |
|      AverageReturn |       -1.19e+03 |
|          StdReturn |            42.5 |
|          MaxReturn |       -1.15e+03 |
|          MinReturn |       -1.27e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        1.56e+04 |
|               Time |            5.94 |
|          Iteration |              12 |

...

avg action -0.2844000752557504. abs 0.35894051989503434
----------------------------------------
|               Time |            16.3 |
|          Iteration |              29 |
|      AverageReturn |       -1.73e+03 |
|          StdReturn |             131 |
|          MaxReturn |       -1.56e+03 |
|          MinReturn |       -1.89e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |         3.6e+04 |
|               Time |            16.3 |
|          Iteration |              29 |
|      AverageReturn |       -1.73e+03 |
|          StdReturn |             131 |
|          MaxReturn |       -1.56e+03 |
|          MinReturn |       -1.89e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |         3.6e+04 |
|               Time |            16.3 |
|          Iteration |              29 |
|

avg action -0.2875314992650927. abs 0.35399720921331695
----------------------------------------
|               Time |            17.6 |
|          Iteration |              32 |
|      AverageReturn |       -1.71e+03 |
|          StdReturn |             157 |
|          MaxReturn |       -1.43e+03 |
|          MinReturn |       -1.86e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        3.96e+04 |
|               Time |            17.6 |
|          Iteration |              32 |
|      AverageReturn |       -1.71e+03 |
|          StdReturn |             157 |
|          MaxReturn |       -1.43e+03 |
|          MinReturn |       -1.86e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        3.96e+04 |
|               Time |            17.6 |
|          Iteration |              32 |
|

avg action -0.25192006856862387. abs 0.3636417189131537
----------------------------------------
|               Time |              19 |
|          Iteration |              35 |
|      AverageReturn |       -1.59e+03 |
|          StdReturn |             119 |
|          MaxReturn |       -1.45e+03 |
|          MinReturn |       -1.81e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        4.32e+04 |
|               Time |              19 |
|          Iteration |              35 |
|      AverageReturn |       -1.59e+03 |
|          StdReturn |             119 |
|          MaxReturn |       -1.45e+03 |
|          MinReturn |       -1.81e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        4.32e+04 |
|               Time |              19 |
|          Iteration |              35 |
|

avg action -0.24588974558391327. abs 0.33423379493977895
----------------------------------------
|               Time |            20.4 |
|          Iteration |              38 |
|      AverageReturn |       -1.63e+03 |
|          StdReturn |             105 |
|          MaxReturn |       -1.53e+03 |
|          MinReturn |       -1.83e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        4.68e+04 |
|               Time |            20.4 |
|          Iteration |              38 |
|      AverageReturn |       -1.63e+03 |
|          StdReturn |             105 |
|          MaxReturn |       -1.53e+03 |
|          MinReturn |       -1.83e+03 |
|          EpLenMean |             200 |
|           EpLenStd |               0 |
| TimestepsThisBatch |         1.2e+03 |
|     TimestepsSoFar |        4.68e+04 |
|               Time |            20.4 |
|          Iteration |              38 |


KeyboardInterrupt: 

In [ ]:
%debug

In [ ]:
env.action_space.sample()